In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from urllib.parse import urljoin
from urllib.request import urlopen
from tqdm import tqdm_notebook
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 네이버 뉴스 가져오기 및 제외글자 추출

In [98]:
headers= {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
        'referer':'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100'}

url1 = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=018&aid=0004617815'
url2 = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=032&aid=0003003068'
url3 = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=015&aid=0004322450'

res = req.get(url1, headers = headers)
soup = bs(res.text, 'html.parser')



scripts = soup.findAll('script')
summarys = soup.findAll('strong',class_ = 'media_end_summary')
photos= soup.findAll('span', class_='end_photo_org')
As = soup.findAll('a')
spans = soup.findAll('div > span', class_= 'u_word_dic')
tds = soup.select('tbody > tr > td')
strongs = soup.select('div > strong')
divs = soup.findAll('div', class_ = 'ab_sub_heading')
explains = soup.select('tr + tr > td')

#스크립트
for script in scripts:
    script.extract() 

#처음에 나오는 기사요약
for summary in summarys :
    summary.extract()
    
#사진 밑에 글
for photo in photos :
    photo.extract()

#사진 밑에 글(2)
for explain in explains :
    explain.extract()

#a 태그
for A in As :
    A.extract()

# span태그    
for span in spans :
    span.extract()

#td태그
for td in tds :
    td.extract()
    
#strong 태그
for strong in strongs :
    strong.extract()
    
#div 태그
for div in divs :
    div.extract()

    
titles= soup.select('h3#articleTitle')    #기사 제목
date = soup.find('span', class_ = 't11').text #기사입력 시간
body = soup.find_all("div", class_ = '_article_body_contents') #기사본문


title = titles[0].text
body = body[0].text.strip()
print("기사 제목 : ",title)
print("\n기사입력 시간 :",date)
print("\n기사 본문 :",body)

기사 제목 :  美연준, 정크본드까지 매입.."할수 있는 건 다 한다"

기사입력 시간 : 2020.04.10. 오후 5:52

기사 본문 : [이데일리 원다연 기자] 미국 중앙은행인 연방준비제도(연준·Fed)가 코로나19로 인한 경기 충격에 대응해 파격적인 유동성 공급을 이어가고 있다. 지난 금융위기 당시 경험을 바탕으로 관련 절차를 정비해 신속하고 선제적인 대응에 나설 수 있다는 분석이다. 미 연준은 9일(현지시간) 중소기업 등을 지원하는 2조3000억달러(한화 약 2800조원) 규모 추가 대책을 발표했다. ‘메인스트리트 대출 프로그램(MSLP)’을 통해 중소기업에 6000억달러를 투입한다. 직원 1만명 이하·매출 25억달러 이하 기업을 대상으로 한 최대 4년 만기 대출과 직원 급여를 측면 지원하는 ‘급여보호프로그램’(PPP)을 가동하는 한편, 회사채·지방채 매입을 본격화해 실물 경제를 지원 사격하기로 했다.구체적으로 고용인 1만명 이하 중소기업에 대한 신규 대출지원에 6000억달러, 지방정부가 발행하는 단기지방채 직접 매입에 5000억달러 규모의 긴급대출제도가 신설된다. 또 고용인 500명 이하 소기업 대상 급여보호프로그램 대출 지원을 위한 긴급대출제도도 신설했다.앞서 발표됐던 회사채 및 자산유동화증권(ABS) 시장 안정을 위한 지원 대상과 규모는 대폭 확대됐다. 회사채 매입을 위한 ‘프라이머리 마켓 기업 신용 기구’(PMCCF) 및 ‘세컨더리 마켓 기업 신용 기구’(SMCCF)를 통한 지원 규모는 기존 2000억달러에서 7500억달러로 늘어났다. 특히 지원 대상을 기존 투자등급에서 일부 투기등급까지 확대했다. 지난 3월22일까지 ‘BBB-’ 이상의 투자등급 채권만 대상으로 했지만 이후 신용등급 강등으로 ‘BB-’ 이상의 투기등급으로 떨어진 이른바 ‘타락 천사(Fallen Angel)’ 기업 회사채까지도 지원 대상에 포함한다는 것이다. 개인 소비자 금융을 지원하는 ‘자산담보부증권 대출 기구’(TALF)를 통한 지원 대상은 오토론, 학자금대출 등을 자산으로

# 문장분리

In [99]:
#온점 기준으로 split
tmp = []
for i in range(len(body.split('다.'))) :
    tmp.append(body.split('다.')[i])

    
#연속된 온점으로 인한 빈 리스트 삭제    
tmp_li = []    
tmp = pd.Series(tmp)

for i in range(len(tmp)) :
    if len(tmp[i]) != 0 :
        tmp_li.append(tmp[i])
        
        
# 문장끝에 온점 다시 추가       
for i in range(len(tmp_li)) :
    tmp_li[i] = tmp_li[i]+'다.'
    

# Series형태로 변환
tmp_li = pd.Series(tmp_li)
 


#쌍따옴표 안에 있는 온점 split 한 것을 다시 함쳐줌
odd = [1,3,5,7,9]
try :
    for i in range(len(tmp_li)) :
        if tmp_li[i].count('"') in odd :
            tmp_li[i] = tmp_li[i]+tmp_li[i+1]
            tmp_li = tmp_li.drop(i+1)         
except :
    pass



#특수문자 쌍따옴표 안에 있는 온점 split 한 것을 다시 함쳐줌
try :
    for i in range(len(tmp_li)) :
        if tmp_li[i].count('“') + tmp_li[i].count('”') in odd :
            tmp_li[i] = tmp_li[i]+tmp_li[i+1]
            tmp_li = tmp_li.drop(i+1)         
except :
    pass


#index reset 적용
tmp_li.reset_index(drop = True, inplace=True)

#strip 적용
for i in range(len(tmp_li)) :
    tmp_li[i] = tmp_li[i].strip()

#1문장 초과, 3문장 이하일 때 마지막 문장 drop
if len(tmp_li) >1 and len(tmp_li) <=3  :
    tmp_li = tmp_li.drop(len(tmp_li)-1)

    
    
#sents에 tmp_li 를 넣어줌
sents = tmp_li


In [100]:
sents = tmp_li
sents.reset_index(drop = True, inplace = True)

In [101]:
#문장단위로 분리된 기사
for i in range(len(sents)) :
    print('[',i+1,']\n',sents[i])

[ 1 ]
 [이데일리 원다연 기자] 미국 중앙은행인 연방준비제도(연준·Fed)가 코로나19로 인한 경기 충격에 대응해 파격적인 유동성 공급을 이어가고 있다.
[ 2 ]
 지난 금융위기 당시 경험을 바탕으로 관련 절차를 정비해 신속하고 선제적인 대응에 나설 수 있다는 분석이다.
[ 3 ]
 미 연준은 9일(현지시간) 중소기업 등을 지원하는 2조3000억달러(한화 약 2800조원) 규모 추가 대책을 발표했다.
[ 4 ]
 ‘메인스트리트 대출 프로그램(MSLP)’을 통해 중소기업에 6000억달러를 투입한다.
[ 5 ]
 직원 1만명 이하·매출 25억달러 이하 기업을 대상으로 한 최대 4년 만기 대출과 직원 급여를 측면 지원하는 ‘급여보호프로그램’(PPP)을 가동하는 한편, 회사채·지방채 매입을 본격화해 실물 경제를 지원 사격하기로 했다.
[ 6 ]
 구체적으로 고용인 1만명 이하 중소기업에 대한 신규 대출지원에 6000억달러, 지방정부가 발행하는 단기지방채 직접 매입에 5000억달러 규모의 긴급대출제도가 신설된다.
[ 7 ]
 또 고용인 500명 이하 소기업 대상 급여보호프로그램 대출 지원을 위한 긴급대출제도도 신설했다.
[ 8 ]
 앞서 발표됐던 회사채 및 자산유동화증권(ABS) 시장 안정을 위한 지원 대상과 규모는 대폭 확대됐다.
[ 9 ]
 회사채 매입을 위한 ‘프라이머리 마켓 기업 신용 기구’(PMCCF) 및 ‘세컨더리 마켓 기업 신용 기구’(SMCCF)를 통한 지원 규모는 기존 2000억달러에서 7500억달러로 늘어났다.
[ 10 ]
 특히 지원 대상을 기존 투자등급에서 일부 투기등급까지 확대했다.
[ 11 ]
 지난 3월22일까지 ‘BBB-’ 이상의 투자등급 채권만 대상으로 했지만 이후 신용등급 강등으로 ‘BB-’ 이상의 투기등급으로 떨어진 이른바 ‘타락 천사(Fallen Angel)’ 기업 회사채까지도 지원 대상에 포함한다는 것이다.
[ 12 ]
 개인 소비자 금융을 지원하는 ‘자산담보부증권 대출 기구’(TALF)를 통한 지원 대상은 오토론, 학자금대

In [102]:
# konlpy의 코모란 이용하여 단어 그래프 생성 함수정의

from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True) 
    words = [w for w in words if ('/NNP' in w or '/NNG' in w)] #고유명사와 일반명사만 뽑음
    return words

# 기사의 키워드 추출

In [103]:
from textrank import KeywordSummarizer

#키워드 뽑기
keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)

#문장 내에서 뽑은 keyword 중 중요성 기준 상위 30개
keywords = keyword_extractor.summarize(sents, topk = 30) 
keywords

[('대출/NNG', 5.200035036620394),
 ('지원/NNG', 3.628948911864922),
 ('연준/NNG', 2.9617300292424726),
 ('긴급/NNG', 2.848039647758799),
 ('재무부/NNP', 2.1614784781389402),
 ('대상/NNG', 2.0612935001972654),
 ('정부/NNG', 2.0070418119293936),
 ('제도/NNG', 1.6690831832973063),
 ('장관/NNG', 1.590059020002371),
 ('도입/NNG', 1.590059020002371),
 ('조치/NNG', 1.539595364014414),
 ('사전/NNG', 1.4661566594198177),
 ('대응/NNG', 1.3435439210993838),
 ('권한/NNG', 1.327472740217221),
 ('금융/NNG', 1.2910507681115648),
 ('중앙은행/NNP', 1.28294339775492),
 ('회사채/NNP', 1.2708903981551214),
 ('당시/NNG', 1.2449820764615849),
 ('법/NNG', 1.2103064554750476),
 ('이하/NNG', 1.1928718075339915),
 ('기업/NNG', 1.168706178909256),
 ('매입/NNG', 1.1576908000144095),
 ('규모/NNG', 1.1575014120443048),
 ('바탕/NNG', 1.0598478241836569),
 ('결정/NNG', 1.0431568355225487),
 ('협의/NNG', 1.0431568355225487),
 ('마련/NNG', 1.0431568355225487),
 ('한은/NNP', 1.0431568355225487),
 ('신용/NNG', 1.012021226699403),
 ('유동성/NNP', 0.990043504196294)]

In [104]:
# 한글자인 keyword 제외, 단어만 추출
keywords_list = []
for i in range(len(keywords)) :
    if len(keywords[i][0].split('/')[0]) != 1 :
        keywords_list.append(keywords[i][0].split('/')[0])
        
#두 글자 이상인 keyword 중에서 차례대로 상위 5개만 뽑음
keywords_list = keywords_list[:5]
keywords_list

['대출', '지원', '연준', '긴급', '재무부']

# 기사의 요약문장 추출

In [105]:
#본문의 문장 수에 따른 요약문장 수(topk) 결정

if len(sents) <= 1 :
    topk = 1
elif len(sents) <= 2 :
    topk = 2
elif len(sents) <= 4 : 
    topk = 3
elif len(sents) < 9 :
    topk = 4
elif len(sents) < 19 :
    topk = 5
elif len(sents) < 29 :
    topk = 6
elif len(sents) < 39 :
    topk = 7
else : 
    topk = 8

In [106]:
from textrank import KeysentenceSummarizer

#뉴스는 첫 문장이 중요하기 때문에 첫문장이 다른 문장보다 10배 중요하다고 설정
#다른 문장들 중에서도 첫 문장과 비슷할수록 상대적인 중요도가 더 커진다

bias = np.ones(len(sents))
bias[0] = 5 
bias[-1] = 0.01

summarizer = KeysentenceSummarizer(tokenize = komoran_tokenize, min_sim = 0.3)
keysents = summarizer.summarize(sents, topk = topk, bias = bias)
keysents.sort() # 원래 중요도의 순서대로 나왔던 것을 본문기준 순서로 바꿔줌

print("기존 기사의 문장 수 : ",len(sents)) 
print("요약된 문장 수 : ",topk) 
print("이 기사의 키워드 : ",keywords_list)
print('*'*30,'요약된 기사','*'*30,
     '\n',title,
     '\n',date)
for i in range(len(keysents)) :
    print('[',i+1,']\n', keysents[i][2])

기존 기사의 문장 수 :  27
요약된 문장 수 :  6
이 기사의 키워드 :  ['대출', '지원', '연준', '긴급', '재무부']
****************************** 요약된 기사 ****************************** 
 美연준, 정크본드까지 매입.."할수 있는 건 다 한다" 
 2020.04.10. 오후 5:52
[ 1 ]
 [이데일리 원다연 기자] 미국 중앙은행인 연방준비제도(연준·Fed)가 코로나19로 인한 경기 충격에 대응해 파격적인 유동성 공급을 이어가고 있다.
[ 2 ]
 구체적으로 고용인 1만명 이하 중소기업에 대한 신규 대출지원에 6000억달러, 지방정부가 발행하는 단기지방채 직접 매입에 5000억달러 규모의 긴급대출제도가 신설된다.
[ 3 ]
 또 고용인 500명 이하 소기업 대상 급여보호프로그램 대출 지원을 위한 긴급대출제도도 신설했다.
[ 4 ]
 연준의 이같은 유동성 공급 조치는 코로나19 긴급제정법(CARES법)의 보증지원을 바탕으로 한다.
[ 5 ]
 한은은 “연준은 사전에 재무부 장관 협의를 통해 마련된 긴급대출제도 도입 요건 및 대출 조건에 따라 재무부 장관의 사전 승인을 통해 긴급대출제도 도입을 결정했다”며 “금융위기 당시 경험 등을 바탕으로 중앙은행의 준재정적 활동에 대한 법률 및 절차를 미리 보완하여 신속하게 대응할 수 있었던 것”이라고 평가했다.
[ 6 ]
 전문가들은 한은 역시 연준과 같은 방식의 정부가 보증하는 특수목적법인을 통한 회사채 매입 방안이 필요하다면서도 지원 대상 등의 기준 설정이 관건이라고 강조했다.


In [107]:
#기존 기사 각 문장 별 중요도
summarizer.R

array([1.72839499, 0.65209131, 0.88853541, 0.53337654, 1.08847295,
       1.47230907, 1.20841561, 1.02842513, 0.79725198, 0.81068309,
       0.89500922, 0.88438212, 1.0088838 , 1.39365869, 0.96712257,
       1.20330165, 1.11788931, 1.10348204, 1.01128998, 1.47740652,
       0.8451157 , 1.19599081, 0.43702905, 1.35617982, 0.84113712,
       0.66425935, 0.38990618])